# Pattern Recognition hw4

In [1]:
from hmmlearn import hmm
import numpy as np 
import pandas as pd
from textblob import TextBlob
from tqdm import tqdm

In [61]:
ts_data = pd.read_csv('test_final.csv')[:5000]
tr_data = pd.read_csv('train_final.csv')[:5000]

In [62]:
tr_ok = tr_data[tr_data.Sentiment == 0]
tr_ff = tr_data[tr_data.Sentiment == 1]

n_ok = len(tr_ok)
n_ff = len(tr_ff)

In [63]:
tr_ok_f = tr_ok.SentimentText
tr_ff_f = tr_ff.SentimentText

tr_f    = tr_data.SentimentText
tr_l    = tr_data.Sentiment
ts_f    = ts_data.SentimentText
ts_l    = ts_data.Sentiment

In [64]:
prior_ok = np.log(n_ok/(n_ok+n_ff))
prior_ff = np.log(n_ff/(n_ok + n_ff))

In [65]:
class Embed():
    def __init__(self, tr_features):
        self.dict = {}
        for f in tqdm(tr_features):
            for word in TextBlob(f).words:
                if str(word) in self.dict:
                    self.dict[str(word)] += 1
                else:
                    self.dict[str(word)] = 1
        self.freqs(200)
        
    def freqs(self , n_words):
        freq = []
        for word in self.dict:
            freq.append((word, self.dict[word]))
        freq.sort(key=lambda x:x[1] , reverse=True)

        n_words = min(n_words , len(freq))
        self.embeding = {}
        for i in range(n_words):
            self.embeding[freq[i][0]] = i
        self.embeding["BAUM"] = n_words
        return freq

    def trns_f(self, f):
        ret = []
        for word in TextBlob(f).words:
            if word in self.embeding:
                ret.append([self.embeding[word]])
            else:
                ret.append([self.embeding["BAUM"]])
        return ret
    
    def trns_df(self, tr_features):
        X = []
        lengths = []
        for f in tqdm(tr_features):
             em = self.trns_f(f)
             if len(em) == 0:
                continue
             X = X + em
             lengths.append(len(em))
        return np.array(X).reshape(-1, 1) , np.array(lengths).reshape(-1,)
        

In [66]:
embed_ok = Embed(tr_ok_f)
embed_ff = Embed(tr_ff_f)

100%|██████████| 1445/1445 [00:00<00:00, 1936.69it/s]


In [67]:
model_ok = hmm.MultinomialHMM(n_components=20)
model_ff = hmm.MultinomialHMM(n_components=20)

In [54]:
X, lengths = embed_ok.trns_df(tr_ok_f)
model_ok.fit(X, lengths)

100%|██████████| 1392/1392 [00:00<00:00, 1615.45it/s]


MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=20,
               n_iter=10, params='ste',
               random_state=RandomState(MT19937) at 0x7F23E00A5360,
               startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
               verbose=False)

In [55]:
X, lengths = embed_ff.trns_df(tr_ff_f)
model_ff.fit(X, lengths)

100%|██████████| 608/608 [00:00<00:00, 1822.74it/s]


MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=20,
               n_iter=10, params='ste',
               random_state=RandomState(MT19937) at 0x7F23E00A5360,
               startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
               verbose=False)

In [58]:
class bayes_log():
    def __init__(self, p0, p1 , m0, m1 , e0 , e1):
        self.p0 = p0
        self.m0 = m0
        self.e0 = e0

        self.p1 = p1
        self.m1 = m1
        self.e1 = e1

    
    def pred(self, f):
        f0 = self.e0.trns_f(f)
        f1 = self.e1.trns_f(f)
        if len(f0)== 0 or len(f1)==0:
            print("kossher")
            return 0
        if self.p0 + self.m0.score(f0) >= self.p1 + self.m1.score(f1):
            return 0
        else:
            return 1
    
    def predict(self, fs):
        pred = []
        for f in fs:
            pred.append(self.pred(f))
        return pred

    def acc(self, pred , ls):
        pred = np.array(pred)
        ls   = np.array(ls)

        return sum(pred == ls)/len(pred)




In [59]:
clf = bayes_log(prior_ok, prior_ff, model_ok, model_ff, embed_ok , embed_ff)

In [60]:
clf.acc(clf.predict(tr_f) , tr_l)

kossher
kossher
kossher


0.4435